## Zillow Housing Data Time Series Projections

## Setup

In [12]:
import pandas as pd
import matplotlib 
import numpy as np

import itertools
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX

import sys
sys.path.insert(0, '../code')

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
%matplotlib inline
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}
matplotlib.rc('font', **font)

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',50)

# EDA

## Load Data

In [2]:
# df = pd.read_csv('zillow_data.csv')

In [4]:
df = pd.read_csv('data/Zip_zhvi_bdrmcnt_2_uc_sfrcondo_tier_0.33_0.67_sm_sa_mon.csv')

In [5]:
df

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1996-01-31,...,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2021-01-31,2021-02-28
0,61639,0,10025,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,218945.0,...,1147187.0,1146710.0,1147090.0,1147833.0,1148717.0,1148276.0,1152108.0,1160117.0,1164604.0,1164231.0
1,84654,1,60657,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,195076.0,...,405010.0,405461.0,406698.0,408293.0,410351.0,412261.0,414306.0,416267.0,417997.0,419795.0
2,61637,2,10023,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,NaN,...,1721974.0,1720842.0,1717028.0,1714060.0,1714273.0,1719429.0,1721641.0,1722443.0,1719270.0,1718401.0
3,91982,3,77494,Zip,TX,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,NaN,...,263993.0,264834.0,265177.0,266047.0,267064.0,268704.0,270651.0,273364.0,275817.0,278966.0
4,84616,4,60614,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,210976.0,...,455800.0,455937.0,457165.0,458964.0,461415.0,463713.0,466099.0,468604.0,470797.0,473676.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26763,59376,34430,4109,Zip,ME,ME,Portland,Portland-South Portland,Cumberland County,NaN,...,437448.0,439683.0,440365.0,443039.0,446706.0,452621.0,458311.0,464683.0,471777.0,476052.0
26764,66881,34430,21405,Zip,MD,MD,Annapolis,Baltimore-Columbia-Towson,Anne Arundel County,NaN,...,655468.0,658822.0,667879.0,675408.0,687068.0,700051.0,714711.0,725972.0,731296.0,737869.0
26765,61694,34430,10118,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,NaN,...,2831506.0,2812227.0,2797581.0,2771802.0,2752092.0,2717823.0,2671871.0,2622981.0,2574374.0,2540750.0
26766,95186,34430,86343,Zip,AZ,AZ,Crown King,Prescott,Yavapai County,NaN,...,160611.0,159760.0,158093.0,158567.0,160055.0,162540.0,163833.0,164567.0,165747.0,167391.0


# Step 2: Data Preprocessing

# Step 3: EDA and Visualization

In [10]:


# NOTE: if you visualizations are too cluttered to read, try calling 'plt.gcf().autofmt_xdate()'!

# Step 4: Reshape from Wide to Long Format

In [8]:
# def melt_df(df):
#     cheese_list = []
#     for i in list(df.RegionID):
#         one_zip_df = df.loc[df['RegionID'] == i]

# #     cheese = pd.concat([one_zip_df.RegionID,one_zip_df.iloc[:,8:]],axis = 1)
#         cheese = melt_data(one_zip_df)
# #     cheese = pd.melt(cheese, id_vars =['RegionID'])
#     cheese_list.append(cheese)
#     cheese_df = pd.concat(cheese_list, axis = 0)

# cheese_df.variable = pd.to_datetime(cheese_df.variable)

In [9]:
def melt_data(df):
    melted = pd.melt(df, id_vars=['RegionID', 'RegionName', 'City', 'State', 'StateName', 'Metro', 'CountyName', 'SizeRank', 'RegionType'], var_name='date')
    melted['date'] = pd.to_datetime(melted['date'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
#     return melted.groupby('time').aggregate({'value':'mean'})
#     return melted.groupby('date')
    return melted

In [10]:
cheese_df = melt_data(df)

In [11]:
cheese_df

,RegionID,RegionName,City,State,StateName,Metro,CountyName,SizeRank,RegionType,date,value
0,61639,10025,New York,NY,NY,New York-Newark-Jersey City,New York County,0,Zip,1996-01-31,232527.0
1,84654,60657,Chicago,IL,IL,Chicago-Naperville-Elgin,Cook County,1,Zip,1996-01-31,208780.0
2,61637,10023,New York,NY,NY,New York-Newark-Jersey City,New York County,2,Zip,1996-01-31,184917.0
3,91982,77494,Katy,TX,TX,Houston-The Woodlands-Sugar Land,Harris County,3,Zip,1996-01-31,199929.0
4,84616,60614,Chicago,IL,IL,Chicago-Naperville-Elgin,Cook County,4,Zip,1996-01-31,281068.0
...,...,...,...,...,...,...,...,...,...,...,...
9163581,62532,12345,Central Point,OR,OR,Medford,Jackson County,34430,Zip,2021-02-28,164643.0
9163582,90986,75599,Texarkana,TX,TX,Texarkana,Bowie County,34430,Zip,2021-02-28,50320.0
9163583,58379,1470,Groton,MA,MA,Boston-Cambridge-Newton,Middlesex County,34430,Zip,2021-02-28,393842.0
9163584,58117,822,Choudrant,LA,LA,Ruston,Lincoln Parish,35187,Zip,2021-02-28,192702.0


In [12]:
columns = ['RegionID', 'CountyName', 'date', 'value']
cheese_df_cut = cheese_df[columns]

In [13]:
sf_df = cheese_df_cut[cheese_df_cut.CountyName == 'San Francisco County']
sf_df.drop('CountyName', axis=1, inplace=True)

In [14]:
la_df = cheese_df_cut[cheese_df_cut.CountyName == 'Los Angeles County']
la_df.drop('CountyName', axis=1, inplace=True)

In [15]:
sf_df

,RegionID,date,value
13,97564,1996-01-31,333727.0
92,97565,1996-01-31,285314.0
387,97576,1996-01-31,298437.0
586,97567,1996-01-31,245673.0
857,97569,1996-01-31,333281.0
...,...,...,...
9139666,97581,2021-02-28,1818997.0
9139849,97563,2021-02-28,1114306.0
9142029,97610,2021-02-28,1392945.0
9145560,97566,2021-02-28,1274126.0


In [16]:
ids = list(set(sf_df.RegionID))
keys = ["sf_cheese_df_" + i for i in list(map(str,ids))]
sf_df_list= []

for i in ids:
    sf_df_list.append(sf_df[sf_df.RegionID == i])
    sf_df_list[-1].drop('RegionID', inplace = True, axis = 1)
    sf_df_list[-1].columns = ['date', 'value']
    sf_df_list[-1].date = pd.to_datetime(sf_df_list[-1].date)
    sf_df_list[-1].set_index('date', inplace = True)
    sf_df_list[-1] = sf_df_list[-1].asfreq('M')
    
sf_dict = dict(zip(keys, sf_df_list))

ids = list(set(la_df.RegionID))
keys = ["la_cheese_df_" + i for i in list(map(str,ids))]
la_df_list= []

for i in ids:
    la_df_list.append(la_df[la_df.RegionID == i])
    la_df_list[-1].drop('RegionID', inplace = True, axis = 1)
    la_df_list[-1].columns = ['date', 'value']
    la_df_list[-1].date = pd.to_datetime(la_df_list[-1].date)
    la_df_list[-1].set_index('date', inplace = True)
    la_df_list[-1] = la_df_list[-1].asfreq('M')
    
la_dict = dict(zip(keys,la_df_list))

In [17]:
len(set(la_df.RegionID))

275

In [18]:
len(set(sf_df.RegionID))

25

In [19]:
cutoff = round(len(sf_dict[list(sf_dict.keys())[0]])*.8)

sf_train_dict_list = [sf_dict[i][:cutoff] for i in list(sf_dict.keys())]
sf_train_dict = dict(zip(list(sf_dict.keys()), sf_train_dict_list))

la_train_dict_list = [la_dict[i][:cutoff] for i in list(la_dict.keys())]
la_train_dict = dict(zip(list(la_dict.keys()), la_train_dict_list))

# ARIMA Modeling

## Manual search of different paramters

In [28]:
p = q = range(0, 3)
pdq = list(itertools.product(p, [1], q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, [1], q))]
print('Examples of parameter for SARIMA...')
for i in pdq:
    for s in seasonal_pdq:
        print('SARIMAX: {} x {}'.format(i, s))

Examples of parameter for SARIMA...
SARIMAX: (0, 1, 0) x (0, 1, 0, 12)
SARIMAX: (0, 1, 0) x (0, 1, 1, 12)
SARIMAX: (0, 1, 0) x (0, 1, 2, 12)
SARIMAX: (0, 1, 0) x (1, 1, 0, 12)
SARIMAX: (0, 1, 0) x (1, 1, 1, 12)
SARIMAX: (0, 1, 0) x (1, 1, 2, 12)
SARIMAX: (0, 1, 0) x (2, 1, 0, 12)
SARIMAX: (0, 1, 0) x (2, 1, 1, 12)
SARIMAX: (0, 1, 0) x (2, 1, 2, 12)
SARIMAX: (0, 1, 1) x (0, 1, 0, 12)
SARIMAX: (0, 1, 1) x (0, 1, 1, 12)
SARIMAX: (0, 1, 1) x (0, 1, 2, 12)
SARIMAX: (0, 1, 1) x (1, 1, 0, 12)
SARIMAX: (0, 1, 1) x (1, 1, 1, 12)
SARIMAX: (0, 1, 1) x (1, 1, 2, 12)
SARIMAX: (0, 1, 1) x (2, 1, 0, 12)
SARIMAX: (0, 1, 1) x (2, 1, 1, 12)
SARIMAX: (0, 1, 1) x (2, 1, 2, 12)
SARIMAX: (0, 1, 2) x (0, 1, 0, 12)
SARIMAX: (0, 1, 2) x (0, 1, 1, 12)
SARIMAX: (0, 1, 2) x (0, 1, 2, 12)
SARIMAX: (0, 1, 2) x (1, 1, 0, 12)
SARIMAX: (0, 1, 2) x (1, 1, 1, 12)
SARIMAX: (0, 1, 2) x (1, 1, 2, 12)
SARIMAX: (0, 1, 2) x (2, 1, 0, 12)
SARIMAX: (0, 1, 2) x (2, 1, 1, 12)
SARIMAX: (0, 1, 2) x (2, 1, 2, 12)
SARIMAX: (1, 1, 0) 

In [37]:
zipcodes = []
param_list = []
param_seasonal_list = []
aic_list = []


# for train in [sf_train_dict[i] for i in list(sf_train_dict.keys())]:
for zipcode, train in sf_train_dict.items():
    for param in pdq:
        for param_seasonal in seasonal_pdq:
#             try:
                mod=SARIMAX(train,
                             order=param,
                             seasonal_order=param_seasonal,
                             enforce_stationarity=False,
                             enforce_invertibility=False)
                zipcodes.append(zipcode[-5:])
                param_list.append(param)
                param_seasonal_list.append(param_seasonal)
                aic_list.append(mod.fit().aic)
                print(aic_list[-1])
#             except: 
#                 print('Oops!')
#                 continue

4713.169841856549
4439.441350731824
4188.208476997037
4444.816949032707
4425.756903072921
4185.257541793078
4201.613117840266
4203.696469140751
4180.583762961309
4596.978528862114
4323.424455745766
4066.9672760107264
4360.485195601678
4313.266116251336
4068.732107629222
4110.2810604837305
4111.88138064795
4068.3328704529677
4502.37303269031
4209.500427791174
3939.0246321020545
4280.036662485937
4206.030595523776
3937.852050380966
4009.2858249757014
4008.570334286458
3936.1684349834736
4495.202125140217
4222.705780074495
3914.382656357502
4180.948950615
4181.357557507601
3936.6593552570826
3877.8334969508287
3922.3725886437405
3924.0785531496736
4478.480266252882
4206.004471034783
3925.30773938502
4183.968646070221
4165.682647840588
3920.418669866226
3924.079467439251
3925.566267554787
3908.0276914882656
4449.044061415082
4170.913223463095
3894.0185948390963
4177.051736286618
4121.157776249935
3891.2394802405533
3915.4601794682085
3916.899585960592
3878.8432780645835
4478.461275900625
4

/Users/wax/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "


3919.9905320028797
3899.9677393711318


/Users/wax/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "


3901.477013836464


...

3416.8941258377963
3223.0451880807586


/Users/wax/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "


3255.412962965659


/Users/wax/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "


4018.952350180609
3801.7635038918243
3511.5955962499256
3221.3060761393535
3506.4650503503753
3483.8250680373058
3215.7489714312187
3215.363732678764
3216.8112027767943
3195.2848415743983


In [34]:
for zipcode, train in sf_train_dict.items():
    print(zipcode[-5:]);

97557
97558
97559
97560
97562
97563
97564
97565
97566
97567
97568
97569
97570
97571
97572
97575
97576
97577
97578
97581
97585
97586
97587
97588
97610


In [38]:
zipcodes;

['97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97557',
 '97558',
 '97558',
 '97558',
 '97558',
 '97558',
 '97558',
 '97558',
 '97558',
 '97558',
 '97558',
 '97558',
 '97558',
 '97558',
 '97558',
 '97558',
 '97558',
 '97558',
 '97558',
 '97558',


In [39]:
# intialise data of lists.
data = {'zipcodes': zipcodes,
        'param': param_list,
        'param_seasonal': param_seasonal_list,
        'aic': aic_list}

# Create DataFrame
sarimax_details_df = pd.DataFrame(data)

# Print the output.
sarimax_details_df.shape

(2025, 4)

In [40]:
sarimax_details_df

,zipcodes,param,param_seasonal,aic
0,97557,"(0, 1, 0)","(0, 1, 0, 12)",4713.169842
1,97557,"(0, 1, 0)","(0, 1, 1, 12)",4439.441351
2,97557,"(0, 1, 0)","(0, 1, 2, 12)",4188.208477
3,97557,"(0, 1, 0)","(1, 1, 0, 12)",4444.816949
4,97557,"(0, 1, 0)","(1, 1, 1, 12)",4425.756903
...,...,...,...,...
2020,97610,"(2, 1, 2)","(1, 1, 1, 12)",3483.825068
2021,97610,"(2, 1, 2)","(1, 1, 2, 12)",3215.748971
2022,97610,"(2, 1, 2)","(2, 1, 0, 12)",3215.363733
2023,97610,"(2, 1, 2)","(2, 1, 1, 12)",3216.811203


In [ ]:
sarimax_details_df

# Step 6: Interpreting Results